# Analisi multiprocessing

Di seguito alcune considerazioni sull'utilizzo
del multiprocessing per la parallelizzazione di
un algoritmo genetico.


## Motivazioni

L'utilizzo del modulo `multiprocessing` è ciò che
viene considerato lo standard per il calcolo
parallelo in Python, soprattutto se si parla di
task CPU bound.

Permette di aggirare il problema introdotto dal
GIL, il quale non permette il classico paradigma
multithread, possibile in altri linguaggi come C o
C++.

Tramite multiprocessing è dunque possibile non
scendere a compromessi in quanto ad espressività
del codice. Il programmatore è libero di scrivere
il suo algoritmo genetico utilizzato tipi e
strutture dati native di Python e, se necessario
può ricorrere a librerie di terze parti senza
problemi.


### Problematiche

Il multiprocessing di contro non è la cosa più
leggera che ci sia. Di certo non è possibile
pensare ad un approccio in cui i processi vengono
creati e distrutti ad ogni iterazione, in quanto
si genererebbe del dell'overhead sicuramente non
trascurabile.

La scelta di creare un pool di processi worker che
rimane in vita dall'inizio alla fine
dell'esecuzione è dunque una scelta quasi obbligata
e che introduce la necessità di un qualche
meccanismo di sincronizzazione per avviare e
mettere in attesa i processi.

L'altro potenziale limite è l'assenza di memoria
condivisa. Non è dunque possibile avere strutture
dati condivise tramite puntatori o definite
globalmente a cui ogni processi può accedere. Anche
qui la scelta di condividere parti della struttura
dati tramite qualche meccanismo di streaming dati
è praticamente obbligata.


#### Memoria condivisa

Esiste la possibilità di creare un blocco di
memoria condivisa che risiede al di fuori di ogni
worker e al quale è possibile accedere in modo
diretto come si farebbe tramite multithreading.

Ho trovato tuttavia questo meccanismo molto
limitante per l'espressività che vorrebbe in
qualche modo garantire la libreria. Il motivo è
che per riuscire a accedere direttamente al blocco
di memoria lo si deve fare tramite oggetti che
supportano il **Buffer Protocol** di Python, come
ad esempio _numpy array_ o _bytearray_.

Questo limita molto i tipi e le strutture dati
che si vogliono impiegare e ci spinge ad un
approccio più classico in cui possiamo manipolare
solo array numerici o simili.

Non sarebbe quindi possibile avere cromosomi
dalla struttura complessa ma soprattutto
personalizzata.


## Modello di calcolo parallelo

Il modello di calcolo proposto si offre di operare
in parallelo nelle fasi di crossover, mutazione e
valutazione.

Una volta selezionati gli individui per la
riproduzione, si suddivide la lista che li contiene
in $W$ chunk uguali, dove $W$ è il numero di
processi worker. Ciascun chunk viene poi inviato
ai worker.

Per la condivisione ho optato per un meccanismo
basato su code (`multiprocessing.Queue`) di
comunicazione. Ogni processo possiede due code, una
per la ricezione dati, l'altra per l'invio.

![queue](images/queue.svg)

Le code permettono di implementare in modo molto
semplice il paradigma _produttore-consumatore_,
fornendo due metodi principali (`put` e `get`)
che permettono rispettivamente di

- Inserire un elemento nella coda. Se questa è
  piena il processo si blocca finché non vi è
  uno slot libero.
- Estrarre un elemento dalla coda. Nel caso in cui
  questa sia vuota ci si blocca in attesa che un
  elemento venga inserito.

Per velocizzare ulteriormente l'invio e la
ricezione dati ho fatto uso della libreria
`asyncio`, la quale, tramite la sintassi
`async`/`await` permette di effettuare operazioni
I/O bound in modo asincrono.


### Prestazioni della coda

Per valutare la bontà del modello ho preso in considerazione il tempo
necessario a

- inserire un singolo individuo in coda.
- effettuare le operazioni di crossover, mutazione e valutazione per un singolo
  individuo.
- effettuare le due operazioni precedenti ma per un gruppo di individui.


In [ ]:
import sys

# chromosome length
chromosome = [i for i in range(500)]
print(f"chromosome size: {sys.getsizeof(chromosome) / 1024.0:.2f} KB")

Passiamo ora a costruire 3 diverse popolazioni, rispettivamente da $10.000$,
$20.000$ e $50.0000$ individui.


In [ ]:
import pandas as pd

population_sizes = [10000, 20000, 50000]
populations = [[chromosome for _ in range(size)] for size in population_sizes]

populations_df = {
    "individuals": [size for size in population_sizes],
    "size (KB)": [sys.getsizeof(pop) / 1024.0 for pop in populations],
}

populations_df = pd.DataFrame(populations_df)
populations_df

Per ognuna delle popolazioni andiamo a valutare il tempo medio di inserimento
ed estrazione di un individuo nella coda.


In [ ]:
import multiprocessing as mp
import time

import pandas as pd

dfs = []

buffer = mp.Queue()
single_timings = {"size": [], "put time (ms)": [], "get time (ms)": []}
for p, size in zip(populations, population_sizes):
    for _ in range(size):
        start = time.perf_counter()
        buffer.put(chromosome)
        put_end = time.perf_counter()
        buffer.get()
        get_end = time.perf_counter()

        single_timings["size"].append(size)
        single_timings["put time (ms)"].append((put_end - start) * 1000.0)
        single_timings["get time (ms)"].append((get_end - put_end) * 1000.0)

single_timings = pd.DataFrame(single_timings)
single_timings.groupby("size").mean()

Vediamo invece il tempo totale di inserimento ed estrazione di tutti gli
individui uno per uno.


In [ ]:
single_timings.groupby("size").sum()

Andiamo ora a valutare il tempo necessario ad inserire ed estrarre l'intera
popolazione in un colpo solo.


In [ ]:
assert buffer.empty()

whole_pop_timings = {"size": [], "put time (ms)": [], "get time (ms)": []}
for pop, size in zip(populations, population_sizes):
    start = time.perf_counter()
    buffer.put(pop)
    put_end = time.perf_counter()
    buffer.get()
    get_end = time.perf_counter()

    whole_pop_timings["size"].append(size)
    whole_pop_timings["put time (ms)"].append((put_end - start) * 1000.0)
    whole_pop_timings["get time (ms)"].append((get_end - put_end) * 1000.0)

whole_pop_timings = pd.DataFrame(whole_pop_timings)
whole_pop_timings

Andiamo a comparare i tempi medi di inserimento ed estrazione di un singolo
individuo con i tempi necessari ad inserire ed estrarre l'intera popolazione.


In [ ]:
whole_pop_timings.groupby("size").sum() / single_timings.groupby("size").mean()

Prendiamo ad esempio la popolazione da $50.000$ individui, con un peso totale
di circa $433$ KB. Si nota subito che occupa almeno 100 volte la memoria
occupata da un singolo cromosoma. Ad ogni modo, il tempo di estrazione
dell'intera popolazione non è 100 volte superiore ma circa 23 volte superiore.

Compariamo ora il tempo necessario ad inserire ed estrarre l'intera popolazione
un individuo alla volta, con il tempo necessario ad inserirla ed estrarla in un
colpo solo.


In [ ]:
single_timings.groupby("size").sum() / whole_pop_timings.groupby("size").sum()

Risulta chiaro il vantaggio che si inserendo più dati alla volta ma meno volte
rispetto ad inserire pochi byte per volta ma molte volte.

Per quanto riguarda invece il singolo inserimento o estrazione dalla coda,
lavorare con meno byte risulta più vantaggioso ma il miglioramento è di gran
lunga più marcato nel caso in cui si lavori con l'intera popolazione in blocco.


#### Suddivisione della popolazione in chunk

Al fine di migliorare il modello che ho implementato, è interessante vedere
la differenza in prestazioni quando si suddivide la popolazione in chunk.

Per ognuna delle popolazioni ho valutato chunk di dimensioni pari alla
popolazione stessa e poi via via più piccoli dividendo per 2.


In [ ]:
import math

chunk_df = {"size": [], "chunksize": [], "put time (ms)": [], "get time (ms)": []}

for size, pop in zip(population_sizes, populations):
    chunksize = size
    for _ in range(6):
        for i in range(0, len(pop), chunksize):
            start = time.perf_counter()
            buffer.put(pop[i : i + chunksize])
            put_end = time.perf_counter()
            buffer.get()
            get_end = time.perf_counter()

            chunk_df["size"].append(size)
            chunk_df["chunksize"].append(chunksize)
            chunk_df["put time (ms)"].append((put_end - start) * 1000.0)
            chunk_df["get time (ms)"].append((get_end - put_end) * 1000.0)

        chunksize = math.ceil(chunksize / 2)

chunk_df = pd.DataFrame(chunk_df)
chunk_df

In [9]:
import matplotlib.pyplot as plt


def plot_chunk(size: int):
    df = chunk_df[chunk_df["size"] == size].sort_values(by="chunksize")
    display(df)
    chunksizes = df["chunksize"].unique()

    mean_put = df.groupby("chunksize").mean()["put time (ms)"]
    mean_get = df.groupby("chunksize").mean()["get time (ms)"]

    tot_put = df.groupby("chunksize").sum()["put time (ms)"]
    tot_get = df.groupby("chunksize").sum()["get time (ms)"]

    plt.figure(figsize=(12, 6))
    plt.title(f"Population size: {size}")
    plt.xlabel("Chunksize")
    plt.ylabel("Operation time")
    # plt.xscale("log")

    plt.plot(chunksizes, mean_put, label=f"mean put time", marker="s")
    plt.plot(chunksizes, mean_get, label=f"mean get time", marker="s")

    plt.plot(chunksizes, tot_put, label=f"total put time", marker="x")
    plt.plot(chunksizes, tot_get, label=f"total get time", marker="x")

    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
plot_chunk(10000)
plot_chunk(20000)
plot_chunk(50000)

Come possiamo vedere dai grafici, i tempi di inserimento rimangono bassi. Sono
i tempi estrazione che si comportano in modo interessante.

Quando si hanno tanti chunk piccoli, il tempo medio di estrazione è basso ma
il tempo totale per estrarre tutta la popolazione tende ad essere più grande.

Quando invece si hanno pochi chunk grandi vediamo che il tempo totale di
estrazione dell'intera popolazione tende a diminuire. Notiamo anche che
mantenere un numero di chunk basso ma comunque diverso da 1 ha prestazioni
molto simili a lavorare con l'intera popolazione in blocco.


### Prestazioni calcolo parallelo

Passiamo alle prestazioni offerte dal modello di calcolo parallelo. Al momento
il problema che sto trattando lavora su una funzione di valutazione della
fitness troppo leggera per apprezzare uno _speed-up_ significativo. Ho aggiunto
quindi un ritardo artificiale tramite un `sleep` di un millisecondo.


In [11]:
import tsp

data = pd.read_csv("datasets/towns_500.csv")
x_coords = data["x"]
y_coords = data["y"]
towns = [tsp.Town(x, y) for x, y in zip(x_coords, y_coords)]

Il modulo ha un API simile alla libreria `DEAP` ma non fa uso di un singolo
metodo `register` per registrare le varie fasi dell'algoritmo.


In [12]:
from ppga import base

toolbox = base.ToolBox()
toolbox.set_fitness_weights(weights=(-1.0,))
toolbox.set_generation(tsp.generate, len(towns))
toolbox.set_selection(tsp.tournament)
toolbox.set_mating(tsp.couples_mating)
toolbox.set_crossover(tsp.cx_one_point_ordered, 0.8)
toolbox.set_mutation(tsp.mut_rotation, 0.2)
toolbox.set_evaluation(tsp.evaluate, towns)
toolbox.set_replacement(tsp.merge)

Abbiamo poi una classe `GeneticSolver` che implementa l'algoritmo genetico
vero e proprio e che prende in input la `ToolBox` precedentemente definita.
